## Setup

In [4]:
import numpy as np
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [5]:
%load_ext autoreload
%autoreload 2
    

# import logging
import torch
from torch import nn
import pytorch_lightning as pl
# import warnings

# warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os

if not os.path.exists('data/transactions_train.csv'):
    ! mkdir -p data
    ! curl -OL https://storage.yandexcloud.net/ptls-datasets/age-prediction-nti-sbebank-2019.zip
    ! unzip -j -o age-prediction-nti-sbebank-2019.zip 'data/*.csv' -d data
    ! mv age-prediction-nti-sbebank-2019.zip data/

## Data preproccessing

In [6]:
import os
import pandas as pd

data_path = 'data/'

source_data = pd.read_csv(os.path.join(data_path, 'transactions_train.csv'))
#source_data = pd.read_csv(os.path.join(data_path, 'transactions.csv'))
#source_data = pd.read_csv(os.path.join(data_path, 'transactions_train_tokenized_20bins.csv'))
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [7]:
import ptls

In [8]:
def bin_numeric(data, params):
    for fe in list(params.keys()):
        num_bins = params[fe] - 1
        quantiles = np.quantile(data[fe], [i/num_bins for i in range(num_bins)])
        for i in range(0, num_bins):
            data.loc[(data[fe] > quantiles[i - 1]) * (data[fe] <= quantiles[i]), fe] = i
        data.loc[data[fe] > quantiles[num_bins -1], fe] = num_bins
        data[fe] = data[fe].astype("int64")
    return data

In [9]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    event_time_transformation='none',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
    return_records=True,
)

data=source_data

In [10]:
from ptls.nn.trx_encoder.glove_embedding import GloveEmbedding

In [11]:
#primary encoding of bin features with bin categories

data = bin_numeric(source_data, {"amount_rur" : 50})

In [12]:
glove_embedding = GloveEmbedding(
        feature_names=["trans_date", "small_group", "amount_rur"],
        calculate_cooccur=True,
        embedding_folder="glove_embeddings",
        glove_params={"alpha" : 0.75, 
                      "x_max" : 5000, 
                      "embedding_size" : 24, 
                      "num_epochs_train" : 50}
    )

In [21]:
#glove_embedding.load()

In [13]:
glove_embedding.fit(data)

train started


  2%|█▋                                                                                 | 1/50 [00:05<04:49,  5.90s/it]

Epoch 0: loss = 14057.688468848433


  4%|███▎                                                                               | 2/50 [00:11<04:30,  5.65s/it]

Epoch 1: loss = 6069.036664531014


...

Epoch 47: loss = 43.572566422965316


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [04:50<00:05,  5.94s/it]

Epoch 48: loss = 43.076340482519775


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [04:56<00:00,  5.92s/it]

Epoch 49: loss = 42.6011365113237


In [14]:
data = glove_embedding.tokenize_data(data)

In [15]:
%%time

dataset = preprocessor.fit_transform(data)

CPU times: total: 42.3 s
Wall time: 45.7 s


In [16]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [17]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [18]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

len(train), len(test)

(24000, 6000)

In [19]:
del dataset

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

### Model definition

In [20]:
import ptls

In [21]:
from functools import partial
from ptls.nn import RnnSeqEncoder, TrxEncoder
from ptls.nn.trx_encoder.trx_encoder_glove import TrxEncoderGlove
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoderGlove(glove_embedding, agg_type="mean"),
    hidden_size=256,
    type='gru',
)

# seq_encoder = RnnSeqEncoder(
#     trx_encoder=TrxEncoder(**trx_encoder_params),
#     hidden_size=256,
#     type='gru',
# )

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

### Data loader

In [22]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200,
        ),
    ),
    train_num_workers=1,
    train_batch_size=256,
)

### Trainer

In [23]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=15,
    accelerator="gpu",
    #devices=1,
    enable_progress_bar=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Training 

In [24]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

logger.version = 27


C:\Users\peter\anaconda3\envs\cv\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 239 K 
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
216 K     Trainable params
23.0 K    Non-trainable params
239 K     Total params
0.959     Total estimated model params size (MB)
C:\Users\peter\anaconda3\envs\cv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increas

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


{'loss': tensor(471.2338), 'seq_len': tensor(112.9271)}
CPU times: total: 27.6 s
Wall time: 1min


### Save sequence encoder for other experiments

In [22]:
torch.save(seq_encoder.state_dict(), "coles-emb.pt")

## Inference 

In [25]:
# embedding inference

from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

train_embeds.shape, test_embeds.shape

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\peter\anaconda3\envs\cv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 94it [00:00, ?it/s]

C:\Users\peter\anaconda3\envs\cv\lib\site-packages\pytorch_lightning\loops\epoch\prediction_epoch_loop.py:173: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 94it [00:00, ?it/s]

(torch.Size([24000, 256]), torch.Size([6000, 256]))

In [26]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 258) (6000, 258)


Obtained embeddings can be used as features for model training

For example:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

sc = 0
for i in range(5):
    embed_columns = [x for x in train_df.columns if x.startswith('embed')]
    x_train, y_train = train_df[embed_columns], train_df['target']
    x_test, y_test = test_df[embed_columns], test_df['target']
    
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    sc += clf.score(x_test, y_test)
sc/5